In [1]:
from fenics import *
from ufl import nabla_div, nabla_grad
import dolfin as df
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt

In [2]:
# Scaled variables
L = 1
W = 0.2
mu = 1
rho = 1
delta = W/L
gamma = 0.4*delta**2
beta = 1.25
lambda_ = beta
g = gamma

In [3]:
# Create mesh and define function space
mesh = BoxMesh(df.Point(0, 0, 0), df.Point(L, W, W), 20, 12, 12)
V = VectorFunctionSpace(mesh, 'Lagrange', 1)

In [4]:
# Define boundary condition
TOL = 1E-14
def clamped_boundary(x, on_boundary):
    return on_boundary and x[0] < TOL
bc = DirichletBC(V, Constant((0, 0, 0)), clamped_boundary)

In [5]:
# Define strain and stress
def epsilon(u):
    return 0.5*(nabla_grad(u) + nabla_grad(u).T)

def sigma(u):
    return lambda_ * nabla_div(u) * Identity(d) + 2*mu*epsilon(u)

In [6]:
# Define variational problem
u = TrialFunction(V)
d = u.geometric_dimension()
v = TestFunction(V)
f = Constant((0, 0, -rho * g))
T = Constant((0, 0, 0))
a = inner(sigma(u), epsilon(v))*dx
L = dot(f, v)*dx + dot(T, v)*ds

In [7]:
# Compute solution
u = Function(V)
solve(a == L, u, bc)

In [8]:
# Plot stress
s = sigma(u) - (1./3)*tr(sigma(u)) * Identity(d)
von_Mises = sqrt(3./2 * inner(s, s))
V = FunctionSpace(mesh, 'P', 1)
von_Mises = project(von_Mises, V)
# plot(von_Mises, title='Stress intensity')
# von_Mises.vector().get_local()
print('max von Mises stress: ', von_Mises.vector().get_local().max())
print('min von Mises stress: ', von_Mises.vector().get_local().min())

max von Mises stress:  0.20744774004
min von Mises stress:  -1.0982668343e-07


In [9]:
# Compute magnitude of displacement
u_magnitude = sqrt(dot(u, u))
u_magnitude = project(u_magnitude, V)
# plot(u_magnitude, 'Displacement magnitude')
print('max u_magnitude', u_magnitude.vector().get_local().max())
print('min u_magnitude', u_magnitude.vector().get_local().min())

max u_magnitude 0.220459163685
min u_magnitude -8.68862119302e-07


In [10]:
u_magnitude.vector().get_local().max(), u_magnitude.vector().get_local().min()

(0.2204591636851311, -8.6886211930159177e-07)